# A Hands-on Introduction to Physics-Informed Neural Networks

**By Atharva Hans and Ilias Bilionis (2021)**  
*Publication Link:* [A Hands-on Introduction to Physics-Informed Neural Networks](https://nanohub.org/resources/handsonpinns)  
*DOI:* 10.21981/FN6Y-NC12

---

### Authors:

**Atharva Hans**
- Role: Graduate Research Assistant
- Affiliation: [Predictive Science Lab](https://www.predictivesciencelab.org/), School of Mechanical Engineering, Purdue University
- Contact: [hans1@purdue.edu](mailto:hans1@purdue.edu)

**Ilias Bilionis**
- Role: Associate Professor
- Affiliation: [Predictive Science Lab](https://www.predictivesciencelab.org/), School of Mechanical Engineering, Purdue University
- Contact: [ibilion@purdue.edu](mailto:ibilion@purdue.edu)

---

### Learning Objectives:

1. Understand how to leverage neural networks for solving ODEs.
2. Grasp the concept of using neural networks for simulator-free forward model evaluations, illustrated through a solid mechanics example.
3. Gain insights into how the structure of a network influences its convergence.
4. Delve into the challenge of spectral bias in Deep Neural Networks (DNNs) and explore potential solutions.

---

### Key References:

- [Artificial Neural Networks for Solving Ordinary and Partial Differential Equations](https://arxiv.org/pdf/physics/9705023.pdf)
- [Deep Residual Learning for Image Recognition](https://arxiv.org/pdf/1512.03385.pdf)
- [On the Spectral Bias of Neural Networks](http://proceedings.mlr.press/v97/rahaman19a/rahaman19a.pdf)
- [Fourier Features Let Networks Learn High Frequency Functions in Low Dimensional Domains](https://arxiv.org/pdf/2006.10739.pdf)


In [ ]:
# Import necessary libraries
import torch
import torch.nn as nn
import numpy as np
import os
import requests
from time import perf_counter
from PIL import Image
import matplotlib.pyplot as plt
from functools import partial
from IPython.display import display

# Determine computation device: Use GPU if available, else CPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def download_file_from_url(url, local_filename=None):
    """
    Downloads a file from the provided URL and saves it to the specified path or current working directory.
    
    Parameters:
    - url (str): The URL of the file to be downloaded.
    - local_filename (str, optional): The path where the downloaded file should be saved.
    
    Returns:
    None
    """
    response = requests.get(url)
    
    # If no filename specified, use the filename from the URL
    if local_filename is None:
        local_filename = os.path.basename(url)
        
    with open(local_filename, 'wb') as file:
        file.write(response.content)

# Download an image of ResNet architecture
download_file_from_url("https://www.dropbox.com/s/2xpxzko3d03onwq/ResNet.png?dl=1", 'ResNet.png')

## Example 1: Solving a Single Ordinary Differential Equation (ODE)

Consider the ode:
$$
\frac{d\Psi}{dx} = f(x, \Psi),
$$
with $x \in [0,1]$ and initial conditions (IC):
$$
\Psi(0) = A.
$$
We write the trial solution by:
$$
\hat{\Psi}(x; \theta) = A + x N(x; \theta),
$$
where $N(x; \theta)$ is a neural network (NN).
The solution is $\hat{\Psi}(x;\theta)$ automatically satisfied the initial conditions.
The loss function we would like to minimize to train the NN is:
$$
L(\theta) = \int_0^1 \left[\frac{d\hat{\Psi}(x;\theta)}{dx} - f(x,\hat{\Psi}(x;\theta))\right]^2dx
$$

In [ ]:
# Define the neural network architecture (N) as described by Lagaris et al. 1997
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.layer1 = nn.Linear(1, 50)
        self.layer2 = nn.Linear(50, 1, bias=False)
    
    def forward(self, x):
        x = self.layer1(x)
        x = nn.Sigmoid()(x)
        x = self.layer2(x)
        return x

# Initialize the neural network
N = NeuralNetwork()

# Given initial condition
A = 0.

# Trial solution function using the neural network
Psi_t = lambda x: A + x * N(x)

# Right-hand side of the differential equation
f = lambda x, Psi: torch.exp(-x / 5.0) * torch.cos(x) - Psi / 5.0

# Loss function for training the neural network
def loss(x):
    """Compute the loss based on the discrepancy between the ODE solution and its approximation."""
    x.requires_grad = True
    outputs = Psi_t(x)
    Psi_t_x = torch.autograd.grad(outputs, x, grad_outputs=torch.ones_like(outputs), create_graph=True)[0]
    return torch.mean((Psi_t_x - f(x, outputs)) ** 2)

First, we will use the method that we find in [Lagaris et al](https://arxiv.org/pdf/physics/9705023.pdf).
Instead of using stochastic optimization, they use a lot of points to estimate the loss integral (We will use 100) and then they just do gradient-based optimization (We will do BFGS).

In [ ]:
# Define optimizer for the neural network (Algorithm as mentioned by Lagaris)
optimizer = torch.optim.LBFGS(N.parameters())

# Define the collocation points (as used by Lagaris)
x = torch.Tensor(np.linspace(0, 2, 100)[:, None])

# Define the closure function for optimization
def closure():
    """Compute the loss and perform backpropagation."""
    optimizer.zero_grad()
    l = loss(x)
    l.backward()
    return l

# Train the neural network using the optimizer
for i in range(10):
    optimizer.step(closure)

# Compare the neural network's approximation to the true solution
xx = np.linspace(0, 2, 100)[:, None]

with torch.no_grad():
    yy = Psi_t(torch.Tensor(xx)).numpy()

yt = np.exp(-xx / 5.0) * np.sin(xx)

# Plotting the results
fig, ax = plt.subplots(dpi=100)
ax.plot(xx, yt, label='True Solution')
ax.plot(xx, yy, '--', label='Neural Network Approximation')
ax.set_title('Comparison of True Solution vs Neural Network Approximation')
ax.set_xlabel('$x$')
ax.set_ylabel('$\Psi(x)$')
ax.legend(loc='best')
plt.show()

Now, let's use stochastic gradient descent to minimize the loss integral.

In [ ]:
# Reinitialize the neural network (N) for fresh training
N = nn.Sequential(
    nn.Linear(1, 50),
    nn.Sigmoid(),
    nn.Linear(50, 1, bias=False)
)

# Define the stochastic optimizer (Adam in this case)
adam_optimizer = torch.optim.Adam(N.parameters(), lr=0.01)

# Define the batch size (number of points to use per iteration) and maximum iterations
n_batch = 5
max_iterations = 1000

print("Training the neural network...")

# Training loop
for iteration in range(max_iterations):
    # Randomly sample n_batch data points from the interval [0, 2]
    x_batch = 2 * torch.rand(n_batch, 1)
    
    # Reset the gradients
    adam_optimizer.zero_grad()
    
    # Compute the loss for the sampled batch
    current_loss = loss(x_batch)
    
    # Perform backpropagation to compute gradients
    current_loss.backward()
    
    # Update the neural network parameters
    adam_optimizer.step()
    
    # Print the progress at regular intervals
    if iteration % 100 == 99:
        print(f"Iteration {iteration + 1} completed.")

print("Training completed.")


In [ ]:
# Generate a range of x-values for evaluating the true solution and its neural network approximation
x_values = np.linspace(0, 2, 100)[:, None]

# Evaluate the neural network's approximation over the generated x-values
with torch.no_grad():
    nn_approximation = Psi_t(torch.Tensor(x_values)).numpy()

# Calculate the true solution over the x-values
true_solution = np.exp(-x_values / 5.0) * np.sin(x_values)

# Plotting the results for comparison
fig, ax = plt.subplots(dpi=100)

# Plot the true solution
ax.plot(x_values, true_solution, label='True Solution')

# Plot the neural network approximation
ax.plot(x_values, nn_approximation, '--', label='Neural Network Approximation')

# Set axis labels and legend
ax.set_xlabel('$x$')
ax.set_ylabel('$\Psi(x)$')
ax.legend(loc='best')
plt.title('Comparison of True Solution vs Neural Network Approximation')
plt.show()

### Questions

+ Change `n_batch` to just 1. Does the algorithm work? Did you have to increase the iterations to achieve the same accuracy?
+ Modify the code, so that you now solve the problem for $x$ between 0 and 5. Play with the `n_batch` and `max_it` until you get a good solution.

## Example 2: Deformation of a compressible neo-Hookean material

+ Consider a neo-Hookean square body defined on $(x,y) \in [0,1]^2$. Let $u(x,y)$ describe the displacement field for this body. This body is subjected to the following displacement boundary conditions:
$$
u_x(0,y) = 0,
$$
$$
u_y(0,y) = 0,
$$
$$
u_x(1,y) = \delta,
$$
$$
u_y(1,y) = 0,
$$
with $\delta$ referring to the applied displacement along the x-direction.


+ For this hyperelastic material, the stored energy $E_b$ in the body can be expressed in as:
$$
E_b(u(x,y)) = \int_{[0,1]^2}\left\{\frac{1}{2}(\sum_{i}\sum_{j}{F_{ij}^2} - 2)- \ln(\det(F)) + 50\ln(\det(F))^2\right\} dxdy,
$$
with,
$$
F = I + \nabla u,
$$
where $I$ is an identity matrix.


+ The final orientation of this body is described by a displacement field that minimizes the stored energy $E_b$.


+ Let's describe the horizontal and the vertical components of displacement field ($u_x$ and $u_y$ respectively) for this body as:
$$
u_x(x,y) = \delta - \delta(1-x) + x(1-x)N_x(x,y;\theta),
$$
and,
$$
u_y(x,y) = x(1-x)N_y(x,y;\theta)
$$
where $N_x(x,y;\theta)$ and $N_y(x,y;\theta)$ are neural networks. Notice $u_x(x,y)$ and $u_y(x,y)$ automatically satisfies the boundary conditions. 


+ The loss function that we would like to minimize to train the NN is:
$$
L(\theta) = E_b(u(x,y))
$$
We will Monte Carlo for estimating the integral.

Next, let's define a few helper functions which will be used throughout this activity. 

### Helper Functions

##### Gradient Function

In [ ]:
def grad(outputs, inputs):
    """
    Compute the gradient of 'outputs' with respect to 'inputs'.
    
    Parameters:
    - outputs (torch.Tensor): Tensor for which the gradient needs to be computed.
    - inputs (torch.Tensor): Tensor with respect to which the gradient is computed.
    
    Returns:
    - torch.Tensor: Gradient of 'outputs' with respect to 'inputs'.
    """
    return torch.autograd.grad(
        outputs, 
        inputs, 
        grad_outputs=torch.ones_like(outputs), 
        create_graph=True
    )[0]

##### Displacement Function

In [ ]:
def u_r(x, delta, network1, network2, scale_network_out):
    """
    Compute the displacement field \( u_r \) given the input tensor `x`, boundary conditions `delta`, and neural networks `network1` and `network2`.
    
    Parameters:
    - x (torch.Tensor): Input tensor containing the X and Y coordinates. Shape: (batch size, 2).
    - delta (torch.Tensor): Direchlet boundary conditions. Typically something like: torch.tensor(([right_edge_displacement, 0.])).
    - network1 (torch.nn.Module): Neural network to compute the horizontal component of the displacement.
    - network2 (torch.nn.Module): Neural network to compute the vertical component of the displacement.
    - scale_network_out (float): Scaling factor for the network output.
    
    Returns:
    - torch.Tensor: Displacement field \( u_r \) combining both horizontal (u_x) and vertical (u_y) displacements.
    """
    
    # Calculate horizontal displacement component
    u_x = delta[0] - delta[0]*(1. - x[:, 0][:, None]) + x[:, 0][:, None] * (1. - x[:, 0][:, None]) * network1(x) / scale_network_out
    
    # Calculate vertical displacement component
    u_y = x[:, 0][:, None] * (1. - x[:, 0][:, None]) * network2(x) / scale_network_out
    
    # Combine horizontal and vertical components
    u_r_combined = torch.cat((u_x, u_y), dim=1)
    
    return u_r_combined

##### Loss Function

In [ ]:
def loss(X, u_r):
    """
    Compute the loss for given input data and displacement function.
    
    Parameters:
    - X (torch.Tensor): Input data tensor. Shape: (batch_size, 2).
    - u_r (function): Displacement function that accepts X as input and returns the displacement.
    
    Returns:
    - torch.Tensor: Computed loss value.
    """
    
    # Enable gradient computation for the input tensor
    X.requires_grad = True
    
    # Calculate the displacement using provided function
    displacement = u_r(X)
    
    # Compute gradient of displacement components with respect to input X
    gradient_u_x = grad(displacement[:, 0], X)[:, None, :]
    gradient_u_y = grad(displacement[:, 1], X)[:, None, :]
    
    # Construct the Jacobian matrix
    J = torch.cat([gradient_u_x, gradient_u_y], 1)
    
    # Define the identity matrix
    I = torch.eye(2).repeat(X.shape[0], 1, 1)
    
    # Calculate the deformation gradient tensor
    F = I + J
    
    # Compute the log determinant of the deformation gradient tensor
    log_det_F = torch.log(F[:, 0, 0] * F[:, 1, 1] - F[:, 0, 1] * F[:, 1, 0])
    
    # Compute the residual for the loss
    residual = (0.5 * (torch.einsum('ijk,ijk->i', F, F) - 2) - log_det_F + 50 * log_det_F ** 2)
    
    # Return the mean of the residuals as the final loss
    return torch.mean(residual)

##### Training Loop

In [ ]:
def train_MultiFieldFracture_seperate_net(net_u, net_v, right_edge_displacement, batch_size, max_iterations, print_frequency, scaling_factor):
    """
    Trains the provided neural networks for multi-field fracture problems with separate networks for u_x and u_y.

    Parameters:
    - net_u (torch.nn.Module): Neural network for horizontal displacement.
    - net_v (torch.nn.Module): Neural network for vertical displacement.
    - right_edge_displacement (float): Boundary condition on the right edge.
    - batch_size (int): Number of samples to use for each iteration.
    - max_iterations (int): Maximum number of training iterations.
    - print_frequency (int): Frequency at which to print the training progress.
    - scaling_factor (float): Scaling factor for the network outputs.

    Returns:
    - list: List of loss values.
    - torch.nn.Module: Trained network for horizontal displacement.
    - torch.nn.Module: Trained network for vertical displacement.
    """
    
    print('\nStarting training with separate networks for u_x and u_y...\n')
    print(f'Right Edge Displacement: {right_edge_displacement:.3f}', 
          f'\tBatch Size: {batch_size}', 
          f'\tMax Iterations: {max_iterations}\n')

    net_u = net_u.to(device)
    net_v = net_v.to(device)

    # Define the boundary condition
    delta = torch.tensor([right_edge_displacement, 0.], device=device)

    # Combine parameters of both networks
    parameters = list(net_u.parameters()) + list(net_v.parameters())

    # Initialize the optimizer
    optimizer = torch.optim.Adam(parameters, lr=1e-4)
    
    start_time = perf_counter()
    loss_values = []
    accumulated_loss = 0.0

    for iteration in range(max_iterations):
        # Sample input data uniformly
        X = torch.distributions.Uniform(0, 1).sample((batch_size, 2)).to(device)

        # Reset gradients
        optimizer.zero_grad()

        # Evaluate the loss
        current_loss = loss(X, partial(u_r, delta=delta, network1=net_u, network2=net_v, scale_network_out=scaling_factor))
        accumulated_loss += current_loss.item()

        # Compute gradients
        current_loss.backward()

        # Update network parameters
        optimizer.step()

        # Print progress at specified frequency
        if (iteration + 1) % print_frequency == 0:
            elapsed_time = perf_counter() - start_time
            avg_loss = accumulated_loss / print_frequency
            print(f'[Iteration: {iteration + 1}] Elapsed Time: {elapsed_time:.2f} secs, Loss: {avg_loss:.4f}')
            loss_values.append(avg_loss)
            accumulated_loss = 0.0

    return loss_values, net_u, net_v

##### Model Capacity

In [ ]:
def model_capacity(net):
    """
    Determine and print the capacity of the neural network.

    Parameters:
    - net (torch.nn.Module): Neural network for which capacity is to be determined.

    Returns:
    - tuple: Number of layers and number of learnable parameters in the model.
    """
    
    # Compute the number of learnable parameters
    number_of_learnable_params = sum(p.numel() for p in net.parameters() if p.requires_grad)
    
    # Determine the number of layers in the model
    num_layers = len(list(net.parameters()))
    
    # Print the results
    print(f"\nThe number of layers in the model: {num_layers}")
    print(f"The number of learnable parameters in the model: {number_of_learnable_params}\n")
    
    return num_layers, number_of_learnable_params

##### Plots

In [ ]:
def plot_loss(loss_values, label_name):
    """
    Plot the loss values over iterations.

    Parameters:
    - loss_values (list): List containing the loss values over iterations.
    - label_name (str): Label for the plot representing the loss values.
    """
    
    ax.plot(100 * np.arange(len(loss_values)), loss_values, label=label_name)
    ax.set_xlabel('Iterations')
    ax.set_ylabel('Loss')
    ax.legend(loc='best')
    
    
def plot_displacement(net_u, net_v, right_edge_displacement, scaling_factor, num_samples=200, marker_size=4):
    """
    Visualize the displacement field using the given neural networks.

    Parameters:
    - net_u (torch.nn.Module): Neural network for horizontal displacement.
    - net_v (torch.nn.Module): Neural network for vertical displacement.
    - right_edge_displacement (float): Boundary condition on the right edge.
    - scaling_factor (float): Scaling factor for the network outputs.
    - num_samples (int, optional): Number of grid points for visualization. Defaults to 200.
    - marker_size (int, optional): Size of the scatter plot markers. Defaults to 4.
    """
    
    # Define the boundary condition
    delta = torch.tensor([right_edge_displacement, 0.])
    
    fig, axes = plt.subplots(1, 2, figsize=(20, 5))
    
    # Create a mesh grid for visualization
    x_vals = torch.linspace(0, 1, num_samples)
    xx, yy = torch.meshgrid(x_vals, x_vals)
    grid_points = torch.cat((xx.reshape(-1)[:, None], yy.reshape(-1)[:, None]), axis=1)

    # Compute displacements using the neural networks
    displacements = u_r(grid_points, delta, net_u, net_v, scaling_factor).detach().numpy()
    
    # Calculate final positions after displacements
    final_positions = grid_points + displacements 
    final_positions = final_positions.numpy()
    
    # Plot horizontal displacements
    sc1 = axes[0].scatter(final_positions[:, 0], final_positions[:, 1], s=marker_size, c=displacements[:, 0], cmap='copper')
    fig.colorbar(sc1, ax=axes[0])
    axes[0].set_title('Horizontal Displacement ($u_x$)')
    
    # Plot vertical displacements
    sc2 = axes[1].scatter(final_positions[:, 0], final_positions[:, 1], s=marker_size, c=displacements[:, 1], cmap='copper')
    fig.colorbar(sc2, ax=axes[1])
    axes[1].set_title('Vertical Displacement ($u_y$)')
    
    plt.tight_layout()

Let's start with displacement ($\delta$) of 0.5, so
$$
u_x(1,y) = 0.5.
$$

<b>Note: Solving this problem for displacement ($\delta$) of 0.5 in FEM requires us to break down displacement in smaller chunks. For example, we would first solve for the displacement field for $\delta=0.1$ then using that displacement field as baseline, we would further solve it for $\delta=0.1$ and so on...(until we reach $\delta=0.5$).
</b> 

In [ ]:
# Configuration for the boundary condition and network output scaling.

# Boundary condition (delta) on the right edge.
# Possible values to try include 0.05.
right_edge_displacement = 0.5

# Scaling factor for the network output. The output is scaled down to ensure 
# the Jacobian's determinant is positive, preserving physical meaning. 
# For instance, a negative determinant would imply a negative volume, 
# which isn't physically meaningful.
scale_network_out = 5

### Part A: Simple Fully Connected Neural Network

Let's begin with a simplest case where we have simple fully connected neural networks for both the horizontal displacement $u_x$ and the vertical displacement $u_y$.

##### Define Model

In [ ]:
def initialize_simple_net():
    """
    Initialize a simple neural network with specific layers and activations.
    
    Returns:
    - torch.nn.Module: Initialized neural network.
    """
    return nn.Sequential(
        nn.Linear(2, 50),
        nn.Sigmoid(),
        nn.Linear(50, 50),
        nn.Sigmoid(),
        nn.Linear(50, 50),
        nn.Sigmoid(),
        nn.Linear(50, 50),
        nn.Sigmoid(),
        nn.Linear(50, 50),
        nn.Sigmoid(),
        nn.Linear(50, 50),
        nn.Sigmoid(),
        nn.Linear(50, 50),
        nn.Sigmoid(),
        nn.Linear(50, 1)
    )

# Initialize networks for u_x and u_y
simple_net1 = initialize_simple_net()
simple_net2 = initialize_simple_net()

# Display model capacities for both networks
print("Model capacity for simple_net1 (u_x):")
model_capacity(simple_net1)

print("Model capacity for simple_net2 (u_y):")
model_capacity(simple_net2)

Here is just a sanity check that we start from a candidate solution that indeed satisfies the boundary conditions:

In [ ]:
plot_displacement(simple_net1, simple_net2,
                  right_edge_displacement, scale_network_out)

##### Train Model

Let's train the network for 5000 iterations and batch size of 10. Training should take around 5-7 minutes.

In [ ]:
loss_list_simple_net, simple_net1, simple_net2 = train_MultiFieldFracture_seperate_net(
    net_u=simple_net1, 
    net_v=simple_net2, 
    right_edge_displacement=right_edge_displacement, 
    batch_size=10, 
    max_iterations=5000, 
    print_frequency=100, 
    scaling_factor=scale_network_out
)

##### Analyze Training Loss and Material Displacement

In [ ]:
# Let's visualize the training loss
figure, ax = plt.subplots(dpi=100)
plot_loss(loss_list_simple_net, 'Simple Net')

if right_edge_displacement==0.05:
    ax.plot(100*np.arange(len(loss_list_simple_net)), 0.006*np.ones(len(loss_list_simple_net)), label='FEM Energy')
elif right_edge_displacement==0.5:
    ax.plot(100*np.arange(len(loss_list_simple_net)), 0.43*np.ones(len(loss_list_simple_net)), label='FEM Energy')
plt.legend(loc='best'); 

In [ ]:
# Let's visualize the body post-training
plot_displacement(simple_net1, simple_net2, right_edge_displacement, scale_network_out)

The above loss might or might not converge (to the FEM solution) depending on the number of iterations we trained the network and on the boundary displacement that we choose. If we run it long enough (~20,000 iterations), it might converge. 

Unlike the first example, where we were trying to solve a simple ODE (and the neural network wasn't very deep), here we have relatively deeper networks which may pose two issues:
+ [Vanishing gradients](https://arxiv.org/pdf/1512.03385.pdf)
+ [Bias towards lower frequency functions](http://proceedings.mlr.press/v97/rahaman19a/rahaman19a.pdf)

Is there anything we can do to speed up the convergence?

### Part B: ResNet Architecture 

In this part let's introduce a ResNet architecture for our neural networks and investigate if we get better convergence. 
ResNet architecture includes skip-connections between the layers which has been [shown](https://arxiv.org/pdf/1512.03385.pdf) to overcome the vanishing gradient problem (and help with convergence).

Figure 1a below shows a schematic of a deep ResNet with a dense input layer, followed by K residual blocks and an output layer. 

Figure 1b shows s single residual block with L layers. Pay attention to how we have a skip-connection from the input $z^{(i-1,0)}$ to the output $z^{(i,0)}$. The skip connections lead to better backpropagation of the loss and helps in better convergence. 

In [ ]:
# Load and display the schematic of a deep ResNet with a caption
resnet_image = Image.open('ResNet.png')

# Display the image with a caption
display(resnet_image)
print("Figure: Schematic of a deep ResNet.")

Here is a `DenseResNet` class:

In [ ]:
class DenseResNet(nn.Module):
    """
    Dense ResNet with options for Fourier feature mappings and tunable beta parameters.

    Parameters:
    ----------
    dim_in : int, default=2
        Network's input dimension.
    dim_out : int, default=1
        Network's output dimension.
    num_resnet_blocks : int, default=3
        Number of ResNet blocks.
    num_layers_per_block : int, default=2
        Number of layers per ResNet block.
    num_neurons : int, default=50
        Number of neurons in each layer.
    activation : torch.nn.Module, default=nn.Sigmoid()
        Non-linear activations function.
    fourier_features : bool, default=False
        Whether to pass the inputs through Fourier mapping.
    m_freqs : int, default=100
        Number of frequencies for Fourier mapping.
    sigma : float, default=10
        Controls the spectrum of frequencies.
    tune_beta : bool, default=False
        Whether to consider the beta parameter in the activation function.

    Methods:
    --------
    forward(x) : torch.Tensor
        Forward pass of the network.
    model_capacity() -> Tuple[int, int]
        Returns the number of layers and parameters in the network.
    """
    
    def __init__(self, dim_in=2, dim_out=1, num_resnet_blocks=3, 
                 num_layers_per_block=2, num_neurons=50, activation=nn.Sigmoid(), 
                 fourier_features=False, m_freqs=100, sigma=10, tune_beta=False):
        super(DenseResNet, self).__init__()

        self.num_resnet_blocks = num_resnet_blocks
        self.num_layers_per_block = num_layers_per_block
        self.fourier_features = fourier_features
        self.activation = activation
        self.tune_beta = tune_beta

        # Beta parameters
        if tune_beta:
            self.beta0 = nn.Parameter(torch.ones(1, 1))
            self.beta = nn.Parameter(torch.ones(num_resnet_blocks, num_layers_per_block))
        else: 
            self.beta0 = torch.ones(1, 1)
            self.beta = torch.ones(num_resnet_blocks, num_layers_per_block)

        # First layer
        input_dim = 2 * m_freqs if fourier_features else dim_in
        self.first = nn.Linear(input_dim, num_neurons)
        nn.init.xavier_uniform_(self.first.weight)
        nn.init.zeros_(self.first.bias)

        # ResNet blocks
        self.resblocks = nn.ModuleList([
            nn.ModuleList([nn.Linear(num_neurons, num_neurons) for _ in range(num_layers_per_block)]) 
            for _ in range(num_resnet_blocks)])
        for block in self.resblocks:
            for layer in block:
                nn.init.xavier_uniform_(layer.weight)
                nn.init.zeros_(layer.bias)

        # Last layer
        self.last = nn.Linear(num_neurons, dim_out)
        nn.init.xavier_uniform_(self.last.weight)
        nn.init.zeros_(self.last.bias)

        # Fourier features
        if fourier_features:
            self.B = nn.Parameter(sigma * torch.randn(dim_in, m_freqs))

    def forward(self, x):
        # Fourier feature transformation
        if self.fourier_features:
            x = torch.cat((torch.cos(x @ self.B), torch.sin(x @ self.B)), dim=1)
        x = self.activation(self.beta0 * self.first(x))

        # ResNet blocks
        for i, block in enumerate(self.resblocks):
            z = x
            for j, layer in enumerate(block):
                z = self.activation(self.beta[i][j] * layer(z))
            x = z + x

        return self.last(x)

    def model_capacity(self):
        """Returns the number of layers and parameters in the network."""
        num_params = sum(p.numel() for p in self.parameters() if p.requires_grad)
        num_layers = len(list(self.parameters()))
        return num_layers, num_params

##### Define Model

In [ ]:
def initialize_dense_resnet():
    """
    Initialize a DenseResNet with the given parameters.
    
    Returns:
    - torch.nn.Module: Initialized DenseResNet model.
    """
    return DenseResNet(dim_in=2, dim_out=1, num_resnet_blocks=3, 
                       num_layers_per_block=2, num_neurons=50, activation=nn.Sigmoid(), 
                       fourier_features=False, m_freqs=100, sigma=10, tune_beta=False)

# Initialize two DenseResNet models
dense_net1 = initialize_dense_resnet()
dense_net2 = initialize_dense_resnet()

# Print the model capacities
print("Model capacity for dense_net1:")
model_capacity(dense_net1)

print("Model capacity for dense_net2:")
model_capacity(dense_net2)

<b>Pay attention: We have the same number of parameters as in Part A</b>. Only difference now is that the network has a ResNet architecture (allows better back propagation of loss). Let's train this model and see if this helps in convergence.

Here is just a sanity check that we start from a candidate solution that indeed satisfies the boundary conditions:

In [ ]:
# Let's visualize the body pre-training and make sure the boundary conditions are satified
plot_displacement(dense_net1, dense_net2, right_edge_displacement, scale_network_out)

##### Train Model

Let's train the network for 5000 iterations and batch size of 10. Training should take around 5-7 minutes.

In [ ]:
# Training configuration and execution
training_params = {
    'net_u': dense_net1,
    'net_v': dense_net2,
    'right_edge_displacement': right_edge_displacement,
    'batch_size': 10,
    'max_iterations': 5000,
    'print_frequency': 100,
    'scaling_factor': scale_network_out
}

loss_list_dense, trained_net1, trained_net2 = train_MultiFieldFracture_seperate_net(**training_params)

##### Analyze Training Loss and Material Displacement

In [ ]:
# Let's visualize the training loss
figure, ax = plt.subplots(dpi=100)
plot_loss(loss_list_simple_net, 'Simple Net')
plot_loss(loss_list_dense, 'ResNet')

if right_edge_displacement==0.05:
    ax.plot(100*np.arange(len(loss_list_dense)), 0.006*np.ones(len(loss_list_dense)), label='FEM Energy')
elif right_edge_displacement==0.5:
    ax.plot(100*np.arange(len(loss_list_dense)), 0.43*np.ones(len(loss_list_dense)), label='FEM Energy')
plt.legend(loc='best');

In [ ]:
# Let's visualize the body post-training
plot_displacement(dense_net1, dense_net2, right_edge_displacement, scale_network_out)

Using ResNet architecture seems to have improved the convergence as compared to the case of the simple network in part A. 

One reason for this is as we make the network deeper, it can hurt the ability to train the network to do well on the training set. ResNet overcomes this issue by introducing skip connections between layers; this helps better backpropagation of loss and better convergence.

Can we further improve this?

### Part C: Fourier Features

In this part let's not only use a ResNet architecture for our neural networks but also introduce a Fourier feature mapping for the input data. 
It has been observed that deeper networks are biased towards low frequency functions (["Over-parameterized networks prioritize learning simple patterns that generalize
across data samples"](http://proceedings.mlr.press/v97/rahaman19a/rahaman19a.pdf)). 

To solve this bias towards the low frequency function, [Tancik et al.](https://arxiv.org/pdf/2006.10739.pdf) show that passing the neural network inputs through a simple Fourier feature mapping not only helps neural network learn high-frequency function, but also, helps achieve faster convergence for high frequency components.  

The create a neural network with Fourier feature, just set the parameter `fourier_features` as `True` in the `DenseResNet` class I defined above:

##### Define Model

In [ ]:
def create_dense_net(fourier_features=False):
    """
    Helper function to create a DenseResNet instance with specified parameters.
    """
    return DenseResNet(
        dim_in=2, 
        dim_out=1, 
        num_resnet_blocks=3,
        num_layers_per_block=2, 
        num_neurons=50, 
        activation=nn.Sigmoid(),
        fourier_features=fourier_features, 
        m_freqs=100, 
        sigma=10, 
        tune_beta=False
    )

Fourier_dense_net1 = create_dense_net(fourier_features=True)
Fourier_dense_net2 = create_dense_net(fourier_features=True)

# Display model capacity
def display_model_capacity(net, name):
    """
    Helper function to display model capacity with network name.
    """
    print(f"Model capacity for {name}:")
    model_capacity(net)
    print("------\n")

display_model_capacity(Fourier_dense_net1, "Fourier_dense_net1")
display_model_capacity(Fourier_dense_net2, "Fourier_dense_net2")

Here is just a sanity check that we start from a candidate solution that indeed satisfies the boundary conditions:

In [ ]:
# Let's visualize the body pre-training and make sure the boundary conditions are satified
plot_displacement(Fourier_dense_net1, Fourier_dense_net2, right_edge_displacement, scale_network_out)

##### Train Model

Let's train the network for 5000 iterations and batch size of 10. Training should take around 5-7 minutes.

In [ ]:
# Training configuration and execution using the refactored approach
training_params_fourier = {
    'net_u': Fourier_dense_net1,
    'net_v': Fourier_dense_net2,
    'right_edge_displacement': right_edge_displacement,
    'batch_size': 10,
    'max_iterations': 5000,
    'print_frequency': 100,
    'scaling_factor': scale_network_out
}

loss_list_dense_fourier, trained_net1_fourier, trained_net2_fourier = train_MultiFieldFracture_seperate_net(**training_params_fourier)

##### Analyze Training Loss and Material Displacement

In [ ]:
# Let's visualize the training loss
figure, ax = plt.subplots(dpi=100)
plot_loss(loss_list_simple_net, 'Simple Net')
plot_loss(loss_list_dense, 'ResNet')
plot_loss(loss_list_dense_fourier, 'Fourier-ResNet')

if right_edge_displacement==0.05:
    ax.plot(100*np.arange(len(loss_list_dense)), 0.006*np.ones(len(loss_list_dense)), label='FEM Energy')
elif right_edge_displacement==0.5:
    ax.plot(100*np.arange(len(loss_list_dense)), 0.43*np.ones(len(loss_list_dense)), label='FEM Energy')

plt.legend(loc='best');

In [ ]:
# Let's visualize the body post-training
plot_displacement(Fourier_dense_net1, Fourier_dense_net2, right_edge_displacement, scale_network_out)

This look great! The loss seems to have converged to the FEM solution pretty quickly. 

### Questions

+ For part A, rerun the code for 20,000 iterations. Does the simple network in part A converge? How does the prediction look?
+ How does batch size affect the convergence? Try different batch sizes (20, 40,...) by changing `batch_size_X` in the `train_MultiFieldFracture_seperate_net` function. Which one works the best? 
+ Change the activation function in the `DenseResNet` class from `nn.Sigmoid()` to `nn.ReLU()` and rerun the code. How does the convergence change?
+ In the `DenseResNet` class, read the docstring to see what `tune_beta=True` does. Set `tune_beta=True` and rerun the code. Do you observe better convergence?